In [ ]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [ ]:
df=pd.read_csv('../input/fake-news-data/train.csv')

In [ ]:
df.head()

In [ ]:
#Dropping the NAN values
df=df.dropna()

In [ ]:
#Getting the Independant features
X= df.drop('label', axis= 1)

#Getting the dependant features
y= df['label']

In [ ]:
X.shape


In [ ]:
y.shape

In [ ]:
import tensorflow as tf
tf.__version__

In [ ]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense #due to classification
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout

In [ ]:
voc_size=10000

# **Onehot Representation**

In [ ]:
m= X.copy()
m.reset_index(inplace= True)

In [ ]:
import nltk
import re
from nltk.corpus import stopwords

Data preprocessing and cleaning -----------

In [ ]:
from nltk.stem.porter import PorterStemmer
corpus=[]
ps= PorterStemmer()

for i in range(len(m)):
    text= re.sub(r'\[^a-zA-Z]', ' ', m['title'][i])
    text= text.lower()
    text= re.sub(r'\d+', ' ', text)
    text= re.sub(r'\s+', ' ', text)
    text= [ps.stem(word)for word in text if not word in stopwords.words('english')]
    text= ''.join(text)
    corpus.append(text)

In [ ]:
onehot= [one_hot(words, voc_size) for words in corpus]
onehot

**Embedding representation**

In [ ]:
sent_length= 20
embed_docs= pad_sequences(onehot, padding= 'pre', maxlen= sent_length)
embed_docs

In [ ]:
len(embed_docs)

In [63]:
#Creating the model

vector_features=40
model= Sequential()
model.add(Embedding(voc_size, vector_features, input_length=sent_length))
model.add(Dropout(0.3))#adding the dropout layer
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))#classification problem
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 20, 40)            400000    
_________________________________________________________________
dropout_4 (Dropout)          (None, 20, 40)            0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dropout_5 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 101       
Total params: 456,501
Trainable params: 456,501
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
len(embed_docs), y.shape

In [ ]:
import numpy as np
X_final=np.array(embed_docs)
y_final=np.array(y)
X_final.shape, y_final.shape

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(X_final, y_final, test_size=0.33, random_state=42)

# **Model Training**

In [64]:
#Finally training
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=64)

Epoch 1/10
192/192 [==============================] - 6s 30ms/step - loss: 0.3588 - accuracy: 0.8301 - val_loss: 0.2338 - val_accuracy: 0.9118
Epoch 2/10
192/192 [==============================] - 5s 28ms/step - loss: 0.1500 - accuracy: 0.9429 - val_loss: 0.1857 - val_accuracy: 0.9258
Epoch 3/10
192/192 [==============================] - 5s 27ms/step - loss: 0.0994 - accuracy: 0.9623 - val_loss: 0.1991 - val_accuracy: 0.9176
Epoch 4/10
192/192 [==============================] - 5s 27ms/step - loss: 0.0702 - accuracy: 0.9771 - val_loss: 0.2087 - val_accuracy: 0.9193
Epoch 5/10
192/192 [==============================] - 5s 27ms/step - loss: 0.0463 - accuracy: 0.9851 - val_loss: 0.2443 - val_accuracy: 0.9105
Epoch 6/10
192/192 [==============================] - 5s 27ms/step - loss: 0.0399 - accuracy: 0.9872 - val_loss: 0.2983 - val_accuracy: 0.9216
Epoch 7/10
192/192 [==============================] - 5s 28ms/step - loss: 0.0301 - accuracy: 0.9900 - val_loss: 0.3159 - val_accuracy: 0.9147

 **Performance Metrics and Accuracy**

In [65]:
y_pred= model.predict_classes(X_test)

In [66]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)# for understaing the better classification and other accuracy parameters

array([[3106,  313],
       [ 166, 2450]])

In [67]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.9206296603148302